# Predição de divergências em notas fiscais usando Python e Machine Learning

Nesse projeto, será criado um modelo de Machine Learning baseado na técnica Support Vector Machine (SVM) para prever se a emissão de uma nota fiscal possuirá alguma divergência ou não baseado no tipo do processo, hora de emissão, dia da semana da emissão e valor da nota.

### Importando as bibliotecas

Antes de tudo, iremos importar as bibliotecas necessárias para nosso projeto:

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

### Coleta de Dados e Análise

Agora, vamos carregar o banco de dados das notas fiscais para o projeto:

In [42]:
notafiscal_dataset = pd.read_csv('notafiscal (3).csv')
notafiscal_dataset.head(5)

,tipo_processo,valor_nota,horario_criacao,dia_semana,divergencia
0,1,201.7,754,4,1
1,1,423.7,707,6,0
2,5,113.0,568,4,0
3,4,119.0,186,7,0
4,2,161.7,656,7,1


In [43]:
notafiscal_dataset.shape

(768, 5)

Agora, iremos obter dados estatísticos sobre o banco:

In [44]:
notafiscal_dataset.describe()

,tipo_processo,valor_nota,horario_criacao,dia_semana,divergencia
count,768.000000,768.000000,768.000000,768.000000,768.000000
mean,2.540365,388.581510,519.687500,3.947917,0.447917
std,1.250162,259.696051,367.335228,1.966445,0.497604
min,1.000000,0.100000,1.000000,1.000000,0.000000
25%,2.000000,187.400000,227.250000,2.000000,0.000000
50%,2.000000,343.800000,455.500000,4.000000,0.000000
75%,3.000000,544.900000,704.250000,6.000000,1.000000
max,5.000000,999.400000,1437.000000,7.000000,1.000000


Analisando a distribuição dos valores de saída:
    
    0 = Não apresenta divergência
    1 = Apresenta divergência

In [45]:
notafiscal_dataset['divergencia'].value_counts()

0    424
1    344
Name: divergencia, dtype: int64

Agora, iremos obter a média dos valores de cada atributo do banco de dados, agrupando por diferentes saídas:

In [46]:
notafiscal_dataset.groupby('divergencia').mean()

,tipo_processo,valor_nota,horario_criacao,dia_semana
divergencia,,,,
0,2.969340,497.035377,365.313679,3.886792
1,2.011628,254.905814,709.962209,4.023256


Analisando a média dos atributos obtida, podemos ver que existe uma correlação entre diferentes variáveis com a saída, como o tipo do processo, valor da nota e horario de criação.

Agora iremos separar os dados da saída e imprimir ambos:

In [48]:
X = notafiscal_dataset.drop(columns = 'divergencia', axis=1)
Y = notafiscal_dataset['divergencia']
print(X)
print(Y)

     tipo_processo  valor_nota  horario_criacao  dia_semana
0                1       201.7              754           4
1                1       423.7              707           6
2                5       113.0              568           4
3                4       119.0              186           7
4                2       161.7              656           7
..             ...         ...              ...         ...
763              3        56.5              726           3
764              2       987.3              101           6
765              4       411.8               77           5
766              1       257.3              696           1
767              2       670.7              218           2

[768 rows x 4 columns]
0      1
1      0
2      0
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: divergencia, Length: 768, dtype: int64


### Padronização dos Dados

Como as variáveis possuem diferentes escalas, devemos padronizá-las para podermos realizar previsões:

In [49]:
scaler = StandardScaler()
X = np.asarray(X)
scaler.fit(X)
standardized_data = scaler.transform(X)
X = standardized_data
X

array([[-1.23293466, -0.72008526,  0.63828672,  0.0265033 ],
       [-1.23293466,  0.13531733,  0.51025481,  1.04422992],
       [ 1.96873505, -1.06186097,  0.13160726,  0.0265033 ],
       ...,
       [ 1.16831762,  0.08946467, -1.20591753,  0.53536661],
       [-1.23293466, -0.50584929,  0.4802899 , -1.50008663],
       [-0.43251723,  1.08704903, -0.82182181, -0.99122332]])

Agora, todos os parâmetros estão em uma escala similar.

### Train Test Split

Nesta parte, precisamos dividir nossos dados em sets de treino e de teste para nosso modelo.

Nossos dados de teste terão 20% do banco de dados e os dados de treino terão 80% do banco de dados.

O parâmetro "stratify" éutilizado para balancear a divisão dos dados de maneira proporcional às saídas.

In [50]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state = 2)
print(X.shape, X_train.shape, X_test.shape)

(768, 4) (614, 4) (154, 4)


### Treinando o modelo

Agora, iremos treinar o modelo com os dados de treino obtidos na última seção:

In [51]:
classifier = svm.SVC(kernel='linear')
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

### Avaliação do modelo

Agora, iremos avaliar o nosso modelo:

In [54]:
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Precisão dos dados de treino: ', training_data_accuracy)

Precisão dos dados de treino:  0.7833876221498371


Com esse resultado podemos perceber que a precisão do modelo é de 78.34%, o que pode ser considerado um bom (mas não ótimo) resultado.

Agora, iremos examinar a precisão do nosso modelo performando em cima de dados que não foram utilizados ainda, os dados de teste:

In [55]:
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Precisão dos dados de teste: ', test_data_accuracy)

Precisão dos dados de teste:  0.8311688311688312


Agora, podemos ver que a precisão do modelo é de 83,11% quando submetido ao set de teste, o que representa uma precisão ainda maior do que a obtida com o set de treinamento. Isso representa um resultado muito bom.

### Criando um sistema de previsões

Agora, iremos criar um sistema capaz de prever com certo grau de precisão se a emissão de uma nota fiscal será sucedida de uma divergência ou não, baseado nos dados de entrada.

Para isso, iremos usar os dados de uma nota fiscal cuja qual já sabemos a saída.

Exemplo: 3,609.2,720,5,0

O 0 no final nos mostra que a emissão dessa nota fiscal não foi sucedida de uma divergência. Obviamente, essa informação não será usada em nosso sistema.

In [57]:
input_data = (3,609.2,720,5)
input_data_array = np.asarray(input_data) # Mudando dados de enrada para formato de array
input_data_reshaped = input_data_array.reshape(1, -1) # Mudando os dados de formato, já que vamos prever só um caso
standardized_data = scaler.transform(input_data_reshaped) # Padronizando os dados para coincidirem com o modelo
prediction = classifier.predict(standardized_data) # Prevendo a saída
if prediction[0] == 0:
    print("Nota fiscal não deve apresentar divergências.")
else:
    print("Nota fiscal deve apresentar divergências.")

Nota fiscal não deve apresentar divergências.
